In [2]:
import requests
import re
from bs4 import BeautifulSoup
import json
import urllib
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, MultiLineString

# Способ поиска всех маршрутов в городе по названию

In [3]:
str_ct = 'Самара'
str_qt = urllib.parse.quote(str_ct)
srch_ct = "https://yandex.ru/maps/?text={}".format(str_qt)

In [4]:
req = requests.get(srch_ct)
bs = BeautifulSoup(req.text, 'html.parser')

In [5]:
alla = bs.find_all('a')
for aa in alla:
    cls = aa.get("class")
    try:
        if 'catalog-entry-point' in cls:
            hrf = aa.get("href")
            break
    except:
        pass
# 
try:
    city_id,city_name = hrf.split("maps/")[1].split("/catalog")[0].split("/")
except:
    city_id,city_name = None,None
    print("Error. Not found")
    pass
#


In [6]:
lnk_tp = "https://yandex.ru/maps/"+ str(city_id) + "/" + str(city_name) + "/transport"

In [ ]:
# def get_city_info(city):
# return 

In [7]:
def find_js(lnk):
    req = requests.get(lnk)
    bs = BeautifulSoup(req.text, 'html.parser')
    all_scrpt = bs.find_all("script")
    for scr in all_scrpt:
        if scr.get('type') == "application/json":
            sc_ind = all_scrpt.index(scr)
            break
    # 
    scrp_txt = all_scrpt[sc_ind].text
    js = json.loads(scrp_txt)
    
    return js

In [8]:
js_tp = find_js(lnk_tp)

In [157]:
def get_ids_all(js_tp):

    lst_rts = []
    i=0
    for i in range(len(js_tp['vehicles'])):
        one_lst = []
        try:
            meta_veh = js_tp['vehicles'][i]['properties']['VehicleMetaData']['Transport']
            lineid = meta_veh['lineId']
            if lineid not in lst_lineid:
                one_lst.append(lineid)
                answ_line = 1
            else:
                answ_line = 0
            try:
                transp_type = meta_veh['type']
                name_rt = meta_veh['name']
            except:
                transp_type,name_rt = None,None
            if answ_line == 1:
                one_lst.append(name_rt)
                one_lst.append(transp_type)
            #
            if (one_lst != []) & (one_lst not in lst_rts):
                lst_rts.append(one_lst)
        except:
            pass
    # 

    return lst_rts

In [154]:
lst_rts = get_ids_all(js_tp)

In [10]:
def get_stop(dct):
    lst=[]
    stop_id = dct['id']
    name = dct['name']
    stop_lon = dct['coordinates'][0]
    stop_lat = dct['coordinates'][1]
    lst = [stop_id,name,stop_lat,stop_lon]
    return lst

In [11]:
def get_dir_data(one_dir,direction,lnkid):
    all_stops=[]
    rt_line=[]
    line_data=[]
    i=0
    lst_stop=[]
    cnt_stp = 0
    line_data = line_data + [lnkid,direction]
    for i in range(len(one_dir)):
        if 'id' in one_dir[i]:
            cnt_stp +=1
            lst_stop = []
            lst_stop = lst_stop + [lnkid,direction,cnt_stp]
            lst_stop = lst_stop + get_stop(one_dir[i])
            all_stops.append(lst_stop)

        if 'points' in one_dir[i]:
            one_part = one_dir[i]['points']
            for j in range(len(one_part)):
                rt_line.append(one_part[j])
        #
    # 
    line_data.append(LineString(rt_line))
    
    return all_stops,line_data

In [141]:
def get_rt_data(city_id, city_name, lnkid):
    
    lnk_rt = 'https://yandex.ru/maps/{}/{}/routes/{}'.format(city_id, city_name, lnkid)
    js_rt = find_js(lnk_rt)
    one_rt=js_rt['masstransitLine']['features']

    for_reestr = []
    line_data=[]

    stop_seq1,line_data1 = get_dir_data(one_rt[0]['features'],0,lnkid)
    line_data.append(line_data1)

    stp_frw = one_rt[0]['properties']['ThreadMetaData']['EssentialStops'][0]['name']
    try:
        stp_bckw = one_rt[0]['properties']['ThreadMetaData']['EssentialStops'][1]['name']
    except:
        stp_bckw = stp_frw
    #
    str_rt_nm = str(stp_frw) + "_" + str(stp_bckw)
    for_reestr.append(str_rt_nm)

    if len(one_rt) == 2:
        stop_seq2,line_data2 = get_dir_data(one_rt[1]['features'],1,lnkid)
        stop_seq = stop_seq1 + stop_seq2
        if line_data2 not in line_data:
            line_data.append(line_data2)
        #
        is_circle = False

    else:
        stop_seq = stop_seq1
        is_circle = True

    # 
    
    for_reestr.append(is_circle)
    return for_reestr, stop_seq, line_data

In [ ]:
def get_length(gdf_line,df_reestr):
    cp_gdfl = gdf_line.copy()
    cp_gdfl = cp_gdfl.to_crs('epsg:32637')
    lst_geo = list(cp_gdfl.geometry)

    lst_length = []
    for line in lst_geo:
        lngth = round(line.length/1000,2)
        lst_length.append(lngth)
    # 
    cp_gdfl['length'] = lst_length

    direct = cp_gdfl[cp_gdfl.direction == 0]
    backw = cp_gdfl[cp_gdfl.direction == 1]

    df_reestr = df_reestr.merge(direct[['line_id', 'length']], how='left',on=['line_id'])
    df_reestr['lenght_frw'] = df_reestr['length']
    del df_reestr['length']
    df_reestr = df_reestr.merge(backw[['line_id', 'length']], how='left',on=['line_id'])
    df_reestr['length_bckw'] = df_reestr['length']
    del df_reestr['length']
    df_reestr['length_bckw'] = df_reestr['length_bckw'].fillna(0)
    
    cp_gdfl = None
    del cp_gdfl
    
    return df_reestr

In [142]:
def make_df_from_url(lst_rts):

    big_reestr=[]
    big_seq=[]
    big_line=[]
    i=0
    for i in range(len(lst_rts)):
    # for i in range(2):
        lnkid = lst_rts[i][0]
        rt_nbr = lst_rts[i][1]
        rt_tp = lst_rts[i][2]
        for_r_2 = [lnkid,rt_nbr,rt_tp]
        for_reestr, stop_seq, line_data = get_rt_data(city_id, city_name, lnkid)
        for_r_2 = for_r_2 + for_reestr
        big_reestr.append(for_r_2)
        big_seq = big_seq + stop_seq
        big_line = big_line + line_data
    # 

    # reestr with all routes
    columns=['line_id','route_number','type_ts','route_long_name','is_circle']
    df_reestr = pd.DataFrame(data=big_reestr,columns=columns)

    # geometry of route
    gdf_line = gpd.GeoDataFrame(data=big_line,columns=['line_id','direction','geometry'])
    gdf_line.crs='epsg:4326'

    # stop_sequence
    columns=['line_id','direction','stop_sequence','stop_id','stop_name','lat','lon']
    df_stop_seq = pd.DataFrame(data=big_seq,columns=columns)


    # get line.length for reestr
    df_reestr = get_length(gdf_line,df_reestr)

    return df_reestr,gdf_line,df_stop_seq

In [139]:
import time

In [143]:
%time df_reestr,gdf_line,df_stop_seq = make_df_from_url(lst_rts)

Wall time: 15.5 s


In [ ]:
# # s = "https://yandex.ru/maps/239/sochi/stops/stop__9973362/"
# city_code = '239'
# city_name = 'sochi'
# stop = 'stop__9973362'

# s0 = 'href="/maps/'+str(city_code)+'/'+city_name+'/routes/'

# s = 'https://yandex.ru/maps/{}/{}/stops/{}/'.format(city_code, city_name, stop)

# url = s

# req = requests.get(url)
# bs = BeautifulSoup(req.text, 'html.parser')

# lst_links = bs.find_all('a')

# lst_routes_links = []
# route_names = []
# titles = []
# for link in lst_links:
#     if s0 in str(link) and link.text != 'Маршруты':
#         lst_routes_links.append('https://yandex.ru'+link.get('href'))
#         route_names.append(link.text)
#         titles.append(link.get('title'))
# # 


# url1 = lst_routes_links[0]
# req = requests.get(url1)
# bs = BeautifulSoup(req.text, 'html.parser')

# scrpt = bs.find_all('script')[-2].text
# js = json.loads(scrpt)

# # stop features
# # js['masstransitLine']['features'][0]['features'][-1]

# all_scrpt = bs.find_all('script')
# for scr in all_scrpt:
#     if scr.get("type") == "application/json":
#         print(all_scrpt.index(scr))
# # 


# ################
# str1 = 'остановка общественного транспорта'
# str_qt = urllib.parse.quote(str1)

# srch = "https://yandex.ru/maps/239/sochi/search/{}".format(str_qt)

# req = requests.get(srch)
# bs = BeautifulSoup(req.text, 'html.parser')

# all_scrpt = bs.find_all("script")


# all_scrpt = bs.find_all("script")
# for scr in all_scrpt:
#     if scr.get('type') == "application/json":
#         print(all_scrpt.index(scr))
# # 

# scr1 = all_scrpt[13].text
# js2 = json.loads(scr1)

# # address
# # js2['searchPreloadedResults']['items'][1]['description']